# Manifold GP Supervised Learning via Precision Matrix

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import numpy as np
import torch
import gpytorch
import matplotlib.pyplot as plt
from mayavi import mlab
from importlib.resources import files
from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from manifold_gp.utils.generate_truth import groundtruth_from_mesh

Select the device: GPU or CPU.

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Load Dataset and Generate Ground Truth

In [3]:
data_path = files('manifold_gp.data').joinpath('dragon10k.stl')
nodes, faces, truth = groundtruth_from_mesh(data_path)
sampled_x = torch.from_numpy(nodes).float().to(device)
sampled_y = torch.from_numpy(truth).float().to(device)
(m, n) = sampled_x.shape

## Apply Noise to the Dataset

In order to simulate real case scenario we can apply a normally distributed noise to our dataset. Here we can consider to apply the noise either to the sampled data points from the manifold or to the ground truth relative to each labeled points.

### Apply noise to the manifold
Setting the variable `manifold_noise` you can apply Gaussian noise to the sampled points from the manifold.

In [4]:
manifold_noise = 0.0
noisy_x = sampled_x + manifold_noise * torch.randn(m, n).to(device)

### Apply noise to the ground truth
Setting the variable `function_noise` you can apply Gaussian noise to the training points.

In [5]:
function_noise = 0.01
noisy_y = sampled_y + function_noise * torch.randn(m).to(device)

## Training Dataset

Create the training dataset. You can set the variable `num_train` to define the number of points within your dataset.

In [6]:
num_train = 500
train_idx = torch.randperm(m)[:num_train]
train_x = noisy_x[train_idx, :]
train_y = noisy_y[train_idx]

## Test Dataset

Create the training dataset. You can set the variable `num_test` to define the number of points within your dataset.

In [7]:
num_test = 50
test_idx = torch.randperm(m)[:num_test]
test_x = noisy_x[test_idx, :]
test_y = noisy_y[test_idx]

## Initialize Kernel

Create the Riemann Matern kernel. `nu` control the kernel smoothness like in the stadard Matern kernel. `neighbors` set the number of nearest neighbors in the KNN-based discrete approximation of the Laplace-Beltrami operator and construction of differentiable eigenfunctions. `modes` set the number of the eigenfunctions used to approximate the Riemann Matern kernel. As for classical *GPyTorch* kernels the Riemann Matern kernel can be decorated using with the *ScaleKernel* to have the signal variance parameter.

In [8]:
nu = 2
neighbors = 50
modes = 20
kernel = gpytorch.kernels.ScaleKernel(RiemannMaternKernel(sampled_x, nu, neighbors, modes))

## Initialize Likelihood and GP Model

Define the type of likelihood and the GP model. The GP model takes as input the training set and and labels, the likelihood and the Riemann Matern Kernel.

In [9]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = RiemannGP(sampled_x, sampled_y, likelihood, kernel).to(device)

## Train Model

Train the GP model using the fast Lanczos precision matrix optimization. `lr` set the learning rate; `iters` defines the number of iteratios; `verbose` can be set to `True` for printing the parameters each iteration.

In [10]:
lr = 1e-2
iters = 1000
verbose = True
model.manifold_informed_train(lr, iters, verbose)

Iteration: 0, Loss: 1839905.875, Noise Variance: 0.007, Signal Variance: 1.000, Lengthscale: 0.368, Epsilon: 0.135
Iteration: 1, Loss: 1758323.375, Noise Variance: 0.007, Signal Variance: 1.006, Lengthscale: 0.371, Epsilon: 0.137
Iteration: 2, Loss: 1680531.125, Noise Variance: 0.007, Signal Variance: 1.013, Lengthscale: 0.374, Epsilon: 0.138
Iteration: 3, Loss: 1606415.000, Noise Variance: 0.007, Signal Variance: 1.019, Lengthscale: 0.377, Epsilon: 0.139
Iteration: 4, Loss: 1535861.375, Noise Variance: 0.007, Signal Variance: 1.025, Lengthscale: 0.380, Epsilon: 0.140
Iteration: 5, Loss: 1468746.750, Noise Variance: 0.007, Signal Variance: 1.032, Lengthscale: 0.383, Epsilon: 0.142
Iteration: 6, Loss: 1404949.625, Noise Variance: 0.007, Signal Variance: 1.038, Lengthscale: 0.387, Epsilon: 0.143
Iteration: 7, Loss: 1344342.500, Noise Variance: 0.007, Signal Variance: 1.044, Lengthscale: 0.390, Epsilon: 0.144
Iteration: 8, Loss: 1286799.875, Noise Variance: 0.007, Signal Variance: 1.051, 

Iteration: 72, Loss: 226764.203, Noise Variance: 0.010, Signal Variance: 1.320, Lengthscale: 0.539, Epsilon: 0.208
Iteration: 73, Loss: 223323.984, Noise Variance: 0.010, Signal Variance: 1.323, Lengthscale: 0.540, Epsilon: 0.209
Iteration: 74, Loss: 219974.250, Noise Variance: 0.010, Signal Variance: 1.325, Lengthscale: 0.542, Epsilon: 0.210
Iteration: 75, Loss: 216713.750, Noise Variance: 0.010, Signal Variance: 1.328, Lengthscale: 0.543, Epsilon: 0.210
Iteration: 76, Loss: 213539.266, Noise Variance: 0.010, Signal Variance: 1.330, Lengthscale: 0.545, Epsilon: 0.211
Iteration: 77, Loss: 210444.531, Noise Variance: 0.010, Signal Variance: 1.332, Lengthscale: 0.546, Epsilon: 0.212
Iteration: 78, Loss: 207429.750, Noise Variance: 0.010, Signal Variance: 1.335, Lengthscale: 0.547, Epsilon: 0.212
Iteration: 79, Loss: 204490.391, Noise Variance: 0.010, Signal Variance: 1.337, Lengthscale: 0.549, Epsilon: 0.213
Iteration: 80, Loss: 201623.609, Noise Variance: 0.010, Signal Variance: 1.340, 

Iteration: 144, Loss: 100525.789, Noise Variance: 0.010, Signal Variance: 1.457, Lengthscale: 0.622, Epsilon: 0.245
Iteration: 145, Loss: 99656.805, Noise Variance: 0.010, Signal Variance: 1.458, Lengthscale: 0.623, Epsilon: 0.246
Iteration: 146, Loss: 98800.148, Noise Variance: 0.010, Signal Variance: 1.460, Lengthscale: 0.624, Epsilon: 0.246
Iteration: 147, Loss: 97954.414, Noise Variance: 0.010, Signal Variance: 1.461, Lengthscale: 0.625, Epsilon: 0.247
Iteration: 148, Loss: 97120.977, Noise Variance: 0.010, Signal Variance: 1.463, Lengthscale: 0.626, Epsilon: 0.247
Iteration: 149, Loss: 96297.664, Noise Variance: 0.010, Signal Variance: 1.464, Lengthscale: 0.627, Epsilon: 0.248
Iteration: 150, Loss: 95486.078, Noise Variance: 0.010, Signal Variance: 1.466, Lengthscale: 0.628, Epsilon: 0.248
Iteration: 151, Loss: 94686.875, Noise Variance: 0.010, Signal Variance: 1.467, Lengthscale: 0.629, Epsilon: 0.248
Iteration: 152, Loss: 93897.398, Noise Variance: 0.010, Signal Variance: 1.468,

Iteration: 221, Loss: 57279.473, Noise Variance: 0.010, Signal Variance: 1.554, Lengthscale: 0.685, Epsilon: 0.274
Iteration: 222, Loss: 56925.156, Noise Variance: 0.010, Signal Variance: 1.555, Lengthscale: 0.686, Epsilon: 0.274
Iteration: 223, Loss: 56574.375, Noise Variance: 0.010, Signal Variance: 1.556, Lengthscale: 0.687, Epsilon: 0.275
Iteration: 224, Loss: 56225.527, Noise Variance: 0.010, Signal Variance: 1.557, Lengthscale: 0.687, Epsilon: 0.275
Iteration: 225, Loss: 55881.582, Noise Variance: 0.010, Signal Variance: 1.558, Lengthscale: 0.688, Epsilon: 0.275
Iteration: 226, Loss: 55541.332, Noise Variance: 0.010, Signal Variance: 1.559, Lengthscale: 0.689, Epsilon: 0.276
Iteration: 227, Loss: 55203.699, Noise Variance: 0.010, Signal Variance: 1.560, Lengthscale: 0.690, Epsilon: 0.276
Iteration: 228, Loss: 54868.262, Noise Variance: 0.010, Signal Variance: 1.561, Lengthscale: 0.690, Epsilon: 0.276
Iteration: 229, Loss: 54537.363, Noise Variance: 0.010, Signal Variance: 1.562, 

Iteration: 293, Loss: 38395.559, Noise Variance: 0.010, Signal Variance: 1.623, Lengthscale: 0.732, Epsilon: 0.296
Iteration: 294, Loss: 38207.438, Noise Variance: 0.010, Signal Variance: 1.624, Lengthscale: 0.733, Epsilon: 0.296
Iteration: 295, Loss: 38016.121, Noise Variance: 0.010, Signal Variance: 1.625, Lengthscale: 0.734, Epsilon: 0.297
Iteration: 296, Loss: 37828.688, Noise Variance: 0.010, Signal Variance: 1.626, Lengthscale: 0.734, Epsilon: 0.297
Iteration: 297, Loss: 37641.781, Noise Variance: 0.010, Signal Variance: 1.627, Lengthscale: 0.735, Epsilon: 0.297
Iteration: 298, Loss: 37455.270, Noise Variance: 0.010, Signal Variance: 1.628, Lengthscale: 0.735, Epsilon: 0.297
Iteration: 299, Loss: 37270.371, Noise Variance: 0.010, Signal Variance: 1.628, Lengthscale: 0.736, Epsilon: 0.298
Iteration: 300, Loss: 37089.094, Noise Variance: 0.010, Signal Variance: 1.629, Lengthscale: 0.737, Epsilon: 0.298
Iteration: 301, Loss: 36907.438, Noise Variance: 0.010, Signal Variance: 1.630, 

...

Iteration: 660, Loss: 10162.319, Noise Variance: 0.011, Signal Variance: 1.850, Lengthscale: 0.897, Epsilon: 0.375
Iteration: 661, Loss: 10134.114, Noise Variance: 0.011, Signal Variance: 1.851, Lengthscale: 0.897, Epsilon: 0.375
Iteration: 662, Loss: 10104.649, Noise Variance: 0.011, Signal Variance: 1.851, Lengthscale: 0.897, Epsilon: 0.375
Iteration: 663, Loss: 10076.914, Noise Variance: 0.011, Signal Variance: 1.852, Lengthscale: 0.898, Epsilon: 0.375
Iteration: 664, Loss: 10046.891, Noise Variance: 0.011, Signal Variance: 1.852, Lengthscale: 0.898, Epsilon: 0.375
Iteration: 665, Loss: 10019.610, Noise Variance: 0.011, Signal Variance: 1.853, Lengthscale: 0.898, Epsilon: 0.375
Iteration: 666, Loss: 9992.536, Noise Variance: 0.011, Signal Variance: 1.853, Lengthscale: 0.899, Epsilon: 0.376
Iteration: 667, Loss: 9962.363, Noise Variance: 0.011, Signal Variance: 1.853, Lengthscale: 0.899, Epsilon: 0.376
Iteration: 668, Loss: 9936.160, Noise Variance: 0.011, Signal Variance: 1.854, Len

Iteration: 732, Loss: 8348.388, Noise Variance: 0.011, Signal Variance: 1.882, Lengthscale: 0.921, Epsilon: 0.387
Iteration: 733, Loss: 8325.959, Noise Variance: 0.011, Signal Variance: 1.883, Lengthscale: 0.921, Epsilon: 0.387
Iteration: 734, Loss: 8306.923, Noise Variance: 0.011, Signal Variance: 1.883, Lengthscale: 0.922, Epsilon: 0.387
Iteration: 735, Loss: 8285.122, Noise Variance: 0.011, Signal Variance: 1.884, Lengthscale: 0.922, Epsilon: 0.387
Iteration: 736, Loss: 8263.563, Noise Variance: 0.011, Signal Variance: 1.884, Lengthscale: 0.922, Epsilon: 0.387
Iteration: 737, Loss: 8240.228, Noise Variance: 0.011, Signal Variance: 1.885, Lengthscale: 0.923, Epsilon: 0.387
Iteration: 738, Loss: 8220.925, Noise Variance: 0.011, Signal Variance: 1.885, Lengthscale: 0.923, Epsilon: 0.388
Iteration: 739, Loss: 8197.109, Noise Variance: 0.011, Signal Variance: 1.885, Lengthscale: 0.923, Epsilon: 0.388
Iteration: 740, Loss: 8177.894, Noise Variance: 0.011, Signal Variance: 1.886, Lengthsca

Iteration: 809, Loss: 6850.497, Noise Variance: 0.011, Signal Variance: 1.914, Lengthscale: 0.946, Epsilon: 0.399
Iteration: 810, Loss: 6838.429, Noise Variance: 0.011, Signal Variance: 1.915, Lengthscale: 0.946, Epsilon: 0.399
Iteration: 811, Loss: 6817.906, Noise Variance: 0.011, Signal Variance: 1.915, Lengthscale: 0.946, Epsilon: 0.399
Iteration: 812, Loss: 6800.949, Noise Variance: 0.011, Signal Variance: 1.915, Lengthscale: 0.947, Epsilon: 0.399
Iteration: 813, Loss: 6784.394, Noise Variance: 0.011, Signal Variance: 1.916, Lengthscale: 0.947, Epsilon: 0.399
Iteration: 814, Loss: 6767.454, Noise Variance: 0.011, Signal Variance: 1.916, Lengthscale: 0.947, Epsilon: 0.399
Iteration: 815, Loss: 6747.595, Noise Variance: 0.011, Signal Variance: 1.917, Lengthscale: 0.947, Epsilon: 0.400
Iteration: 816, Loss: 6732.579, Noise Variance: 0.011, Signal Variance: 1.917, Lengthscale: 0.948, Epsilon: 0.400
Iteration: 817, Loss: 6719.076, Noise Variance: 0.011, Signal Variance: 1.917, Lengthsca

Iteration: 886, Loss: 5682.003, Noise Variance: 0.011, Signal Variance: 1.944, Lengthscale: 0.969, Epsilon: 0.410
Iteration: 887, Loss: 5666.959, Noise Variance: 0.011, Signal Variance: 1.944, Lengthscale: 0.969, Epsilon: 0.410
Iteration: 888, Loss: 5653.506, Noise Variance: 0.011, Signal Variance: 1.945, Lengthscale: 0.969, Epsilon: 0.411
Iteration: 889, Loss: 5641.511, Noise Variance: 0.011, Signal Variance: 1.945, Lengthscale: 0.970, Epsilon: 0.411
Iteration: 890, Loss: 5628.211, Noise Variance: 0.011, Signal Variance: 1.946, Lengthscale: 0.970, Epsilon: 0.411
Iteration: 891, Loss: 5613.504, Noise Variance: 0.011, Signal Variance: 1.946, Lengthscale: 0.970, Epsilon: 0.411
Iteration: 892, Loss: 5601.240, Noise Variance: 0.011, Signal Variance: 1.946, Lengthscale: 0.971, Epsilon: 0.411
Iteration: 893, Loss: 5587.683, Noise Variance: 0.011, Signal Variance: 1.947, Lengthscale: 0.971, Epsilon: 0.411
Iteration: 894, Loss: 5573.177, Noise Variance: 0.011, Signal Variance: 1.947, Lengthsca

Iteration: 958, Loss: 4804.588, Noise Variance: 0.011, Signal Variance: 1.970, Lengthscale: 0.989, Epsilon: 0.421
Iteration: 959, Loss: 4791.787, Noise Variance: 0.011, Signal Variance: 1.971, Lengthscale: 0.990, Epsilon: 0.421
Iteration: 960, Loss: 4783.325, Noise Variance: 0.011, Signal Variance: 1.971, Lengthscale: 0.990, Epsilon: 0.421
Iteration: 961, Loss: 4769.370, Noise Variance: 0.011, Signal Variance: 1.971, Lengthscale: 0.990, Epsilon: 0.421
Iteration: 962, Loss: 4756.608, Noise Variance: 0.011, Signal Variance: 1.972, Lengthscale: 0.990, Epsilon: 0.421
Iteration: 963, Loss: 4751.505, Noise Variance: 0.011, Signal Variance: 1.972, Lengthscale: 0.991, Epsilon: 0.421
Iteration: 964, Loss: 4738.196, Noise Variance: 0.011, Signal Variance: 1.972, Lengthscale: 0.991, Epsilon: 0.421
Iteration: 965, Loss: 4729.676, Noise Variance: 0.011, Signal Variance: 1.973, Lengthscale: 0.991, Epsilon: 0.422
Iteration: 966, Loss: 4717.899, Noise Variance: 0.011, Signal Variance: 1.973, Lengthsca

## Model Evaluation

In [11]:
%%capture
likelihood.eval()
model.eval()

Getting faster predictive distributions using [LOVE](https://arxiv.org/abs/1803.06058). We compute the **mean**, the **standard deviation** and one **posterior sample**. In addition we evaluate the kernel at the first point of our dataset.

In [12]:
%%capture
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = likelihood(model(sampled_x))
    mean = preds.mean
#     std = preds.variance.sqrt()
    posterior_sample = preds.sample()
    kernel_eval = kernel(sampled_x[0, :].unsqueeze(0), sampled_x).evaluate().squeeze()

## Plot Results

In [13]:
with torch.no_grad():
    # Bring data to cpu
    if use_cuda:
        sampled_x = sampled_x.cpu().numpy()
        sampled_y = sampled_y.cpu().numpy()
        train_x = train_x.cpu().numpy()
        kernel_eval = kernel_eval.cpu().numpy()
        posterior_sample = posterior_sample.cpu().numpy()
        mean = mean.cpu().numpy()
#         std = std.cpu()

In [14]:
%%capture
%%bash
jupyter nbextension install --py mayavi --user

In [15]:
%%capture
mlab.init_notebook()
v_options = {'mode': 'sphere','scale_factor': 3e-3, }

### Ground Truth with Sample Training Points

In [16]:
mlab.figure()
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=sampled_y)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Mean

In [17]:
mlab.figure()
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=mean)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Standard Deviation

In [18]:
# mlab.figure()
# mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=sampled_y)
# mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

### One Posterior Sample

In [19]:
mlab.figure()
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=posterior_sample)
mlab.points3d(train_x[:,0], train_x[:,1], train_x[:,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

### Kernel Evaluation

In [20]:
mlab.figure()
mlab.triangular_mesh(nodes[:, 0], nodes[:, 1], nodes[:, 2], faces, scalars=kernel_eval)
mlab.points3d(sampled_x[0,0], sampled_x[0,1], sampled_x[0,2], **v_options)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…